# Общая настройка

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars ./jars/postgresql-42.6.0.jar,./jars/clickhouse-jdbc-0.4.6.jar pyspark-shell'

In [2]:
USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")
HOST = os.getenv("HOST")

PG_NAME = os.getenv("POSTGRES_DB")
PG_PORT = os.getenv("POSTGRES_PORT")

CH_NAME = os.getenv("CLICKHOUSE_DB")
CH_PORT = os.getenv("CLICKHOUSE_PORT")

# Настройка подключения pyspark

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark SQL with PostgreSQL") \
    .getOrCreate()

25/05/24 23:42:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
pg_jdbc_url = f"jdbc:postgresql://{HOST}:{PG_PORT}/{PG_NAME}"
pg_properties = {
    "user": USER,
    "password": PASSWORD,
    "driver": "org.postgresql.Driver"
}

In [5]:
ch_jdbc_url = f"jdbc:clickhouse://{HOST}:{CH_PORT}/{CH_NAME}"
ch_properties = {
    "user": USER,
    "password": PASSWORD,
    "driver": "com.clickhouse.jdbc.ClickHouseDriver"
}

# Витрины

In [6]:
from pyspark.sql.functions import *

In [7]:
def load_table(table_name):
    return spark.read.jdbc(url=pg_jdbc_url, table=table_name, properties=pg_properties)

In [8]:
d_customer = load_table("d_customer")
d_product = load_table("d_product")
d_seller = load_table("d_seller")
d_store = load_table("d_store")
d_sipplier = load_table("d_supplier")

f_sales = load_table("f_sales")

In [12]:
def save_report(df, name: str):
    df.write \
    .mode("overwrite") \
    .option("createTableOptions", """
        ENGINE = MergeTree()
        ORDER BY (product_id)
    """) \
    .jdbc(url=ch_jdbc_url, table=name, properties=ch_properties)

## Продажи по продуктам

### Топ-10 продуктов

In [10]:
top_products = f_sales \
    .join(d_product, f_sales.sale_product_id == d_product.product_id) \
    .groupBy(
        d_product.product_id,
        d_product.product_name,
        d_product.product_category,
        d_product.pet_category,
        d_product.product_brand
    ) \
    .agg(
        sum("sale_quantity").alias("total_quantity_sold"),
        sum("sale_total_price").alias("total_revenue")
    ) \
    .orderBy(desc("total_revenue")) \
    .limit(10)

In [ ]:
save_report(top_products, "1_top10products")

25/05/24 23:44:15 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/24 23:44:15 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/24 23:44:15 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [ba38abcc-4b1f-4489-9ccd-c975120fc051] (2 queries & 0 savepoints) is committed.
25/05/24 23:44:15 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [2c0f3e32-7239-4463-89be-ec632f5b9573] (0 queries & 0 savepoints) is committed.


## Продажи по клиентам

## Продажи по времени

## Продажи по магазинам

## Продажи по поставщикам

## Качество продукции

# Пример кода

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

# 1. Создаем SparkSession
spark = SparkSession.builder \
    .appName("Spark SQL with PostgreSQL and ClickHouse") \
    .getOrCreate()

print(spark.sparkContext._jsc.sc().listJars())

# 2. Создаем DataFrame
data = [
    ("Alice", "Sales", 5000),
    ("Bob", "Sales", 6000),
    ("Charlie", "HR", 4000),
    ("David", "HR", 4500),
    ("Eve", "Sales", 5500)
]
columns = ["Name", "Department", "Salary"]
df = spark.createDataFrame(data, columns)

# Показываем исходный DataFrame
print("Исходные данные:")
display(df.toPandas())

# 3. Записываем данные в PostgreSQL
pg_jdbc_url = "jdbc:postgresql://localhost:5432/db"
properties = {
    "user": "Maxim",
    "password": "12345",
    "driver": "org.postgresql.Driver"
}

# Записываем DataFrame в таблицу 'employees'
df.write.jdbc(url=pg_jdbc_url, table="employees", mode="overwrite", properties=properties)

# 4. Читаем данные из PostgreSQL
df_from_postgres = spark.read.jdbc(url=pg_jdbc_url, table="employees", properties=properties)

print("Данные, прочитанные из PostgreSQL:")
display(df_from_postgres.toPandas())

# 5. Выполняем агрегацию данных
# Группируем по департаменту и считаем общую зарплату
aggregated_df = df_from_postgres.groupBy("Department").agg(sum("Salary").alias("Total_Salary"))

print("Агрегированные данные (сумма зарплат по департаментам):")
pandas_df = aggregated_df.toPandas()
display(pandas_df)

# Записываем данные в ClickHouse через JDBC
ch_jdbc_url = "jdbc:clickhouse://localhost:8123/default"
properties = {
    "driver": "com.clickhouse.jdbc.ClickHouseDriver",
    "user": "Maxim",
    "password": "12345"
}

# Записываем данные в ClickHouse
aggregated_df.write.jdbc(url=ch_jdbc_url, table="default.report_ch", mode="append", properties=properties)

# Читаем данные из ClickHouse
ch_df = spark.read.jdbc(url=ch_jdbc_url, table="default.report_ch", properties=properties)

# Создаем временное view
ch_df.createOrReplaceTempView("report_ch_view")

ch_pandas_df = spark.sql("select Total_Salary from report_ch_view where Department = 'HR' limit 1").toPandas()

print("Агрегированные данные (сумма зарплат по департаментам) из ClickHouse где выбран департамент HR:")
display(ch_pandas_df)

# 6. Останавливаем SparkSession
spark.stop()